# Wykorzystanie pakietu `nltk`

In [36]:
import re
import pandas as pd
import funcs 
import importlib
import unicodecsv as csv
import pickle
import random
from timeit import default_timer as timer
from datetime import datetime, timedelta
import nltk
import nltk.lm
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten
from nltk import word_tokenize, sent_tokenize 
from nltk.lm.preprocessing import padded_everygram_pipeline

importlib.reload(funcs)
importlib.reload(nltk)

<module 'nltk' from 'C:\\Users\\Mag\\Anaconda3\\lib\\site-packages\\nltk\\__init__.py'>

In [8]:
print(nltk.__version__)

3.4.5


In [2]:
sp_joined = pd.read_csv('00_dane/sp_joined.csv',  sep='|', encoding='UTF-8')

Kilka przykładów zastosowania funkcji generującej n-gramy.

In [30]:
test = [word_tokenize(sent_tokenize(sp_joined['speech_rawer'][0])[0])]

In [45]:
list(bigrams(test[0]))

[('Chciałbym', 'kilka'),
 ('kilka', 'zdań'),
 ('zdań', 'powiedzieć'),
 ('powiedzieć', 'na'),
 ('na', 'temat'),
 ('temat', 'skutków'),
 ('skutków', 'tej'),
 ('tej', 'ustawy'),
 ('ustawy', '.')]

In [46]:
training_ngrams, padded_sentences = padded_everygram_pipeline(2, test)
for ngramlize_sent in training_ngrams:
    print(list(ngramlize_sent))
    print()
print('#############')
list(padded_sentences)

[('<s>',), ('Chciałbym',), ('kilka',), ('zdań',), ('powiedzieć',), ('na',), ('temat',), ('skutków',), ('tej',), ('ustawy',), ('.',), ('</s>',), ('<s>', 'Chciałbym'), ('Chciałbym', 'kilka'), ('kilka', 'zdań'), ('zdań', 'powiedzieć'), ('powiedzieć', 'na'), ('na', 'temat'), ('temat', 'skutków'), ('skutków', 'tej'), ('tej', 'ustawy'), ('ustawy', '.'), ('.', '</s>')]

#############


['<s>',
 'Chciałbym',
 'kilka',
 'zdań',
 'powiedzieć',
 'na',
 'temat',
 'skutków',
 'tej',
 'ustawy',
 '.',
 '</s>']

Przetesumy model na jednej wypowiedzi

In [48]:
text = sp_joined['speech_raw'][0]
text

' Panie Marszałku! Wysoki Sejmie! Chciałbym kilka zdań powiedzieć na temat skutków tej ustawy. Moim zdaniem podjęcie tej inicjatywy było jak najbardziej zasadne, konieczne i należy raz na zawsze zamknąć to w naszej historii stanowczym stanowiskiem tego Sejmu. Jestem jednak za stwierdzeniem takim, jakie wnioskuje komisja, że stan wojenny był bezzasadny, że decyzja o wprowadzeniu stanu wojennego została podjęta z naruszeniem prawa. Wysoka Izbo! Przyjęcie konstrukcji innej, że stan wojenny był nielegalny, rodzi nie tylko te konsekwencje, o których tu wszyscy na sali myślimy. Mamy przed oczyma ludzi, którzy w tym czasie cierpieli, którzy w tym czasie byli internowani, którzy w stanie wojennym byli skazywani przez niekiedy psychopatycznych sędziów na wyroki, które nie mają nic wspólnego z wymiarem sprawiedliwości. Ale w tym samym stanie wojennym obowiązywało obostrzone prawo karne, mówiąc najogólniej, i w tym samym czasie skazywani byli także przestępcy pospolici. Jeżeli powiemy, że orzecze

In [49]:
tok_text = [list(map(str.lower, word_tokenize(sent))) 
                  for sent in sent_tokenize(text)]

In [50]:
tok_text

[['panie', 'marszałku', '!'],
 ['wysoki', 'sejmie', '!'],
 ['chciałbym',
  'kilka',
  'zdań',
  'powiedzieć',
  'na',
  'temat',
  'skutków',
  'tej',
  'ustawy',
  '.'],
 ['moim',
  'zdaniem',
  'podjęcie',
  'tej',
  'inicjatywy',
  'było',
  'jak',
  'najbardziej',
  'zasadne',
  ',',
  'konieczne',
  'i',
  'należy',
  'raz',
  'na',
  'zawsze',
  'zamknąć',
  'to',
  'w',
  'naszej',
  'historii',
  'stanowczym',
  'stanowiskiem',
  'tego',
  'sejmu',
  '.'],
 ['jestem',
  'jednak',
  'za',
  'stwierdzeniem',
  'takim',
  ',',
  'jakie',
  'wnioskuje',
  'komisja',
  ',',
  'że',
  'stan',
  'wojenny',
  'był',
  'bezzasadny',
  ',',
  'że',
  'decyzja',
  'o',
  'wprowadzeniu',
  'stanu',
  'wojennego',
  'została',
  'podjęta',
  'z',
  'naruszeniem',
  'prawa',
  '.'],
 ['wysoka', 'izbo', '!'],
 ['przyjęcie',
  'konstrukcji',
  'innej',
  ',',
  'że',
  'stan',
  'wojenny',
  'był',
  'nielegalny',
  ',',
  'rodzi',
  'nie',
  'tylko',
  'te',
  'konsekwencje',
  ',',
  'o',
  

In [61]:
train_data, padded_sents = padded_everygram_pipeline(3, tok_text)

In [62]:
from nltk.lm import MLE
model = MLE(3)

In [63]:
model.fit(train_data, padded_sents)
print(model.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 481 items>


In [64]:
print(model.counts)

<NgramCounter with 3 ngram orders and 3408 ngrams>


In [65]:
model.counts['moim']

3

In [66]:
model.counts[['moim']]['zdaniem']

3

In [67]:
model.counts[['moim','zdaniem']]['podjęcie']

1